In [35]:
# Blackjack game

# Requirements
    # Create a simple text-based BlackJack game
    # One player versus an automated dealer.
    # The player can stand or hit.
    # The player must be able to pick their betting amount.
    # Keep track of the players total money.
    # Alert the player of wins,losses, or busts, etc...
    
# Comments:
    # One player vs. automated dealer
    # No insurance, double-downs or card splits
    # Player cannot add to bankroll once initially defined
    # Deck resets after every round

In [36]:
# Initial imports
from IPython.display import clear_output
import random

In [37]:
# Game class for all game setting up and reset functions
class Game(object):
    
    # Initializing with only 1 player
    def __init__(self):
        global p1, dealer, d, play
        p1 = Player("Player 1")
        dealer = Dealer("Dealer")
        d = Deck()  
        play = Play()
        
    # Starts the game, sets the bankroll and kicks off play
    def start(self, player):
        print "Welcome to Blackjack!"
        player.set_bankroll()
        self.loop(player)
    
    # Loop for each round - facilitiates continued play and cashing out
    def loop(self, player):
        while True:
            play.one_round(player)
            if raw_input("Press any key to play again, or C to cash out.").lower().startswith('c'):
                player.cash_out()
                break
            else:
                self.reset()
                clear_output()
                continue
    
    # Resets a round
    def reset(self):
        # global p1, dealer, d
        dealer.hand = Hand()
        p1.hand = Hand()
        d = Deck()

In [38]:
class Person(object):
    
    # Initializes hand
    def __init__(self, name):
        self.name = name
        self.hand = Hand()

In [39]:
class Dealer(Person):
    
    # Inherits from Person and initializes hand
    def __init__(self, name):
        Person.__init__(self, name)

In [40]:
class Player(Person):
    
    # Inherits fromm Person and initializes hand, as well as Initializes bankroll and bet for player
    def __init__(self, name):
        Person.__init__(self, name)
        self.bankroll = 0
        self.bet = 0

    # Adds money to bankroll
    def add_bankroll(self, amount):
        self.bankroll += amount
    
    # Subtracts money from bankroll
    def subtract_bankroll(self, amount):
        self.bankroll -= amount
    
    # Sets the bankroll
    def set_bankroll(self):
        while True:
            try:
                self.bankroll = int(raw_input("%s, how much is your bankroll? " % self.name))
                break
            except:
                print "Please enter an integer."
                continue
    
    # Allows player to cash out and ends the game
    def cash_out(self):
        print "Your take home amount is: ", self.bankroll
        print "Thank you for playing!"
        # Game ends
        
    # Checks if player has enough money to keep on playing and ends the game if not
    def bankroll_check(self, amount):
        if amount >= 0:
            print "You are good to play!" 
            return True
        else: 
            print "You don't have enough money!"
            return False
    
    # Displays the bankroll
    def display_bankroll(self):
        print "Your balance is: ", self.bankroll
    
    # Notifies player of win and deposits winnings in bankroll
    def win(self):
        print "You win!"
        self.add_bankroll(self.bet)
        self.display_bankroll()
    
    # Notifies player of loss and deposits losses in bankroll
    def lose(self):
        print "You lose!"
        self.subtract_bankroll(self.bet)
        self.display_bankroll()
    
    # Notifies player of tie and pushes bet back to bankroll
    def push(self):
        print "Push!"
        self.display_bankroll()
    
    # Notifies player of blackjack and deposits winnings in bankroll
    def natural(self):
        print "Blackjack!"
        self.add_bankroll(self.bet*1.5)
        self.display_bankroll()        

In [41]:
# Deck class will define the attributes of the deck and the point values
class Deck(object):
    
    # Define a class object attribute for the deck and points
    # Since the suite of a card doesn't matter in Blackjack, each suite is treated exactly the same
    single_deck = [('ace', 11), ('two', 2), ('three', 3), ('four', 4), 
                   ('five', 5), ('six', 6), ('seven', 7), ('eight', 8), 
                   ('nine', 9), ('ten', 10), ('jack', 10), ('queen', 10),
                   ('king', 10)] * 4
    
    # Sets the default deck size to be 8 decks
    def __init__(self, num_decks = 8, shoe = single_deck * 8):
        self.num_decks = num_decks
        self.shoe = Deck.single_deck * num_decks

In [42]:
# This will be a generic hand class used for both the dealer and the player(s)
class Hand(object):

    # initializes cards in the hand, the score and the card count
    def __init__(self):
        self.cards = []
        self.score = 0
        
    # Removes a card tuple from the deck and adds it to your hand
    def draw(self, deck):
        card = random.choice(deck.shoe)
        deck.shoe.pop(deck.shoe.index(card))
        self.cards.append(card)
        self.calc_score()
    
    # Replaces aces valued at 11 in the deck with aces valued at 1
    def ace_replace(self):
        self.cards.pop(self.cards.index(('ace', 11)))
        self.cards.append(('ace', 1))
    
    # returns total score of cards in hand
    def scoring(self):
        self.score = sum(v for k, v in self.cards)

    # calculates score, accounting for ace logic
    def calc_score(self):
        # if there is more than one ace in the deck, value the second, third, forth etc. aces as 1 and calculate the score.
        # if the score is greater than 21, value the first ace as 1
        if self.cards.count(('ace', 11)) >= 1:
            while self.cards.count(('ace', 11)) > 1:
                self.ace_replace()
            self.scoring()
            if self.score > 21:
                self.ace_replace()
        self.scoring()
        
    # Displays hand of cards - length argument is so the dealer's first card can be hidden
    def display(self, hide = 0):
        if hide >= 1:
            print "[ hidden card ]"
        for k, v in self.cards[hide:]:
             print '[', k, ']'     

In [43]:
# This class contains all eLements to do with one round of gameplay
class Play(object):

    # Sets into motion one round of gameplay, checking for bankroll > bet and a blackjack after the initial deal
    def one_round(self, player):
        player.display_bankroll()
        self.set_bet(player)
        if player.bankroll_check(player.bankroll - player.bet):
            self.initial_deal(player)
            if not self.blackjack_check(player):
                self.decision(player)
        else: 
            print "Your bet amount exceeds your bankroll!"

    # Setting a bet 
    def set_bet(self, player):
        while True:
            try:
                player.bet = int(raw_input("How much would you like to bet? "))
                break
            except:
                print "Please enter an integer."
                continue
       
    # Initial two card deal and table display
    def initial_deal(self, player):
        for num in [1, 2]:
            player.hand.draw(d)
            dealer.hand.draw(d)
        self.table(player, 1)

    # Main display - shows both the player's hand and the dealer's hand
    def table(self, player, hide):
        clear_output()
        print "Here are your cards!"
        print "%s's hand:" % player.name
        player.hand.display()
        print "%s's hand:" % dealer.name
        dealer.hand.display(hide)
        
    # Dealer's turn after the player(s) have gone
    def robot(self, player):
        self.table(player, 0)
        while dealer.hand.score < 17:
            dealer.hand.draw(d)
            self.table(player, 0)
        
    # Player's turn
    def decision(self, player):
        while True:
            player_decision = raw_input("Hit or stand? ")
            if player_decision.lower().startswith('h'):
                print "Hit!"
                player.hand.draw(d)
                self.table(player, 1)
                if self.bust_check(player):
                    break
                else:
                    continue
            elif player_decision.lower().startswith('s'):
                print "Stand!"
                self.robot(player)
                self.score_check(player)
                break
            else: 
                print "Please try again."
                continue

    # Checks for a blackjack after the initial two-card deal            
    def blackjack_check(self, player):
        if player.hand.score == 21: 
            self.table(player, 0)
            self.show_score(player)
            if dealer.hand.score != 21:
                player.natural()
                return True
            elif dealer.hand.score == 21:
                player.push()
                return True
        else:
            return False
    
    # Checks for a bust after the player hits  
    def bust_check(self, player):
        if player.hand.score > 21:
            self.table(player, 0)
            self.show_score(player)
            print "Busted!"
            player.lose()
            return True
        else:
            return False
    
    # Checks the score once the player decides to stand
    def score_check(self, player):
        self.show_score(player)
        if player.hand.score <= 21:
            if dealer.hand.score > 21:
                print "Dealer busts!"
                player.win()
            elif player.hand.score > dealer.hand.score:
                player.win()
            elif player.hand.score < dealer.hand.score:
                print "Dealer wins!"
                player.lose()
            elif player.hand.score == dealer.hand.score and dealer.hand.score <= 21:
                player.push()
            else:
                print "This is an unaccounted for scenario! Bet pushed."
                player.push()
        else: 
            print "This is an unaccounted for scenario! Bet pushed."
            player.push()
            
    def show_score(self, player):
        for who in [player, dealer]:
            print "%s score: %i" % (who.name, who.hand.score)

In [46]:
game = Game()
game.start(p1)

Here are your cards!
Player 1's hand:
[ queen ]
[ queen ]
Dealer's hand:
[ three ]
[ six ]
[ nine ]
Player 1 score: 20
Dealer score: 18
You win!
Your balance is:  450.0
Press any key to play again, or C to cash out.c
Your take home amount is:  450.0
Thank you for playing!
